In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

# spacy for lemmatization
import spacy


In [ ]:
#!pip install -r requirements.txt

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

stop_words = nltk.corpus.stopwords.words('dutch')
stop_words.extend(['kb', 'pdf','nationaal','rapporteur'])

#Install Dutch Spacy with python -m spacy download nl_core_news_sm
#!python -m spacy download nl_core_news_sm
import nl_core_news_sm

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
nlp = nl_core_news_sm.load()

# Execute Applications/python 3.7/Install Certificates.command
import nltk
nltk.download('stopwords')

df = pd.read_excel('pdf_pages.xlsx')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oprokopenk001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def clean_data(data):
    # Put string in list
    if type(data) == type('string'):
        data = [data]
    #for sent in data[0:100]:
        #print(sent.dtype)
    
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove special charcters
    pattern = r'[^a-zA-z0-9\s]' if not True else r'[^a-zA-z\s]'
    data = [re.sub(pattern, '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    data = [re.sub("\_", "", sent) for sent in data]
    return data


def parse_data(data):

    data = clean_data(data)

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
        # Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
    
    print(data_words_bigrams)

    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized


## Prepare data for model

In [74]:
# Convert to list
data = df.loc[0:10000].par_id.values.tolist()
data_lemmatized = parse_data(data)

# Do lemmatization keeping only noun, adj, vb, adv

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [76]:
# Do lemmatization keeping only noun, adj, vb, adv

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [81]:
len(corpus)

10001

## LSI

In [ ]:
# Try methods: tf-idf, LSI and LDA
lsi = gensim.models.LsiModel(corpus, id2word=id2word, num_topics=13)

doc = "mensenhandel loverboy"
vec_bow = id2word.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

index = gensim.similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
print(list(enumerate(sims)))

## TF-IDF

In [89]:
tfid = TfidfModel(corpus)
doc = data_lemmatized
total= pd.DataFrame([], columns=['f','s','t'])


KeyboardInterrupt: 

In [95]:
for i in range(9598, len(doc)):
    vec_bow = id2word.doc2bow(doc[i])
    vector = tfid[vec_bow]
    d = pd.DataFrame([], columns=['f','s','t'])
    for val in vector:
        word = id2word.get(val[0])
        d=d.append(pd.DataFrame([[i, word,  val[1]]], columns=['f','s','t']))
    total=total.append(d)

In [96]:
i

10000

In [97]:
total.head()

,f,s,t
0,5218,as,0.081521
0,5218,for,0.095492
0,5218,report,0.049047
0,5218,organisatie,0.054521
0,5218,nederlands,0.040383


In [98]:
total.to_csv('all_keywords2.csv')

In [103]:
total1=pd.read_csv('all_keywords2.csv')
total2=pd.read_csv('all_keywords.csv')
total=total1.append(total2)

In [104]:
total.shape

(1079347, 4)

In [106]:
keywords = total.groupby('f')['t'].nlargest(100)
keywords


f            
0      0         1.000000
1      67        0.266672
       18        0.201015
       45        0.167789
       26        0.167274
       24        0.166881
       17        0.164349
       40        0.160068
       3         0.157836
       31        0.157093
       46        0.151988
       2         0.150675
       8         0.143417
       29        0.139219
       64        0.139217
       49        0.138151
       50        0.136192
       55        0.135500
       53        0.134317
       84        0.131369
       80        0.131369
       69        0.131369
       11        0.124914
       6         0.122917
       42        0.121088
       15        0.121088
       81        0.121088
       59        0.120511
       56        0.119402
       79        0.118868
                   ...   
10000  499963    0.067275
       499957    0.066488
       499873    0.066233
       499924    0.065863
       499962    0.065741
       499912    0.065145
       499964    0.06479

In [160]:
total.head()

,Unnamed: 0,f,s,t
0,0,5218,as,0.081521
1,0,5218,for,0.095492
2,0,5218,report,0.049047
3,0,5218,organisatie,0.054521
4,0,5218,nederlands,0.040383


## Search

In [115]:
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent))
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out

In [126]:
doc1=("kinderen loverboy")

doc1.split()

['kinderen', 'loverboy']

In [130]:
        allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
        texts_out = []
        for sent in doc1.split():
            doc = nlp("".join(sent))
            print(doc)
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])


kinderen
loverboy


In [131]:
doc=texts_out

In [161]:
doc[1][0]

'loverboy'

In [144]:
syn=pd.DataFrame()

In [146]:
for index, val in enumerate(doc):
    try:
        syn[str(index)]=(X_train[np.nonzero(X_train ==(val))[0][0],:])
    except:
        syn[str(index)]=" "


In [166]:
return_res=keywords.f[keywords.s==doc[1][0]]

AttributeError: 'Series' object has no attribute 'f'

In [168]:
return_res.shape

(245,)

In [ ]:
data_lemmatized[0]

In [ ]:
id2word.doc2bow("kinderen rapporteur".split())

In [ ]:
"kinderen rapporteur".split()

In [ ]:
doc = parse_data("kinderen loverboy")
vec_bow = id2word.doc2bow(doc[0])
vec_bow


In [ ]:
doc=pd.DataFrame(np.array([["kinderen loverboy"], ["mensenhandel loverboy"]]))

In [ ]:
doc

In [141]:
training_data_x = pd.read_excel("synonyms.xlsx")
X_train = training_data_x.as_matrix()

C:\Users\oprokopenk001\AppData\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [ ]:
X_train

In [ ]:
try:
    print(X_train[np.nonzero(X_train =='misbruik')[0][0],:])
except:
    print ("no synonyms")



In [ ]:
from nltk.corpus import wordnet
syns = wordnet.synsets("dog")
print(syns)

In [ ]:
from nltk.corpus import wordnet
synonyms = []
antonyms = []

for syn in wordnet.synsets("active"):
    for l in syn.lemmas():
    synonyms.append(l.name())
    if l.antonyms():
    antonyms.append(l.antonyms()[0].name())

    print(set(synonyms))
    print(set(antonyms))

In [71]:
for i in range(0,1000):
    #if type(i)!=type(string)):
    try:
        cl_string = re.sub('\S*@\S*\s?', '', data[i])
        print(cl_string)
        data_cl.append(cl_string)
    except:
        print(i)

<div 
>1    pagina 1 van 5 > retouradres postbus 20301 2500 eh  den haag   aan de voorzitter van de tweede kamer der statengeneraal postbus 20018 2500 ea den haag datum 13 februari 2017 onderwerp algemeen overleg mensenhandel  geachte voorzitter, op 16 februari a.s. staat het algemeen overleg inzake mensenhandel op de agenda van uw kamer. in het kader van dit overleg wil ik de aandacht vragen voor vijf onderwerpen. 1. de beschikbare capaciteit voor de aanpak van mensenhandel en de terugloop van het aantal meldingen bij comensha. 2. signalering mensenhandel bij migranten en vluchtelingen. 3. pilot multidisciplinaire vaststelling slachtofferschap mensenhandel. 4. aanpak criminele uitbuiting kinderen. 5. nationaal verwijsmechanisme ad 1) capaciteit aanpak mensenhandel en terugloop meldingen (mogelijke) slachtoffers bij comensha de aanpak van mensenhandel is een van de prioriteiten van dit kabinet. ook uw kamer toont zich steeds betrokken bij dit onderwerp. desondanks heb ik op 30 augustus

>   pagina 4 van 5 datum 13 februari 2017  ons kenmerk 10008  ad 4 aanpak criminele uitbuiting kinderen in mijn rapport zicht op kwetsbaarheid heb ik de aandacht gevraagd voor de aanpak van criminele uitbuiting van kinderen. in nederland is met het oog op deze aanpak het project 13 ocean’s gestart. in dit project werken het openbaar ministerie, nationale politie en partners in zorg en veiligheidsketen samen in een team om criminele uitbuiting van roma kinderen aan te pakken. ondanks de goede resultaten van dit team is deze samenwerking onlangs gestaakt. als nationaal rapporteur acht ik het van belang dat een team dat op dit belangrijke terrein succes boekt in stand blijft. de opgedane kennis en effectieve aanpak mag niet verloren gaan. ik hoop dat u daar bij de minister van veiligheid en justitie op wil aandringen.  ad 5 het nationaal verwijsmechanisme in mijn negende rapportage mensenhandel heb ik de minister van veiligheid en justitie aanbevolen om een nationaal verwijsmechanisme tot

> pagina 3 van 3 datum 5 april 2017  ons kenmerk 10026 een gecoördineerde aanpak van seksueel geweld tegen kinderen kan niet zonder goede beleidsinformatie. op dit terrein schiet deze nog tekort. zo is het bijvoorbeeld nog altijd onbekend hoe vaak vermoedens in nederland worden gemeld en onderzocht bij veilig thuis, of hoeveel slachtoffers van seksueel geweld jeugdhulp of jeugdbescherming ontvangen, en hoe deze hulp eruitziet. hierdoor kunnen we niet zien of de gekozen aanpak daadwerkelijk effectief is. goed beleid vereist gedegen en onafhankelijke monitoring en evaluatie, zodat in de toekomst kan blijken dat beleid ook bewezen effectief is.  ook online vindt seksueel geweld tegen kinderen in toenemende mate en in grote variëteit plaats. zo heeft een aantal omvangrijke groomingzaken veel aandacht getrokken en zijn sextortionzaken allang geen uitzondering meer. in de periode 2008 tot 2012 steeg het aantal bij het om ingeschreven zaken over kinderpornografie met 45%, en sindsdien neemt d

>   pagina 3 van 4 datum 25 oktober 2017  ons kenmerk 10055  de commissie rechtseenheid voorstaat ter zake van de totstandkoming van oriëntatiepunten, namelijk dat veel voorkomende strafbepalende factoren dienen te worden benoemd en dat aan deze factoren een bepaald gewicht dient te worden toegekend.4 het verdient aanbeveling dat de minister de rechtspraak faciliteert in het uitvoeren van dergelijk onderzoek.  aanbeveling 2 (maak de strafmotivering inzichtelijker) mijn tweede aanbeveling luidt dat in de strafmotivering in ontuchtzaken (ten minste) de waardering van de factoren die tot de straf hebben geleid dienen te worden opgenomen, en dat hun onderlinge weging inzichtelijk dient te worden gemaakt. de minister laat weten dat de raad voor de rechtspraak geen toegevoegde waarde ziet in een dergelijk voorschrift. daarnaast verwijst de minister naar de door strafrechters opgestelde professionele standaarden, die volgens hem een stimulerende rol kunnen spelen bij het meer inzichtelijk mak

>1    pagina 1 van 2 > retouradres postbus 20301 2500 eh  den haag aan de minister van justitie en veiligheid dhr. f. grapperhaus turfmarkt 147 2511 dp den haag  aan de minister van volksgezondheid, welzijn en sport dhr. h. de jonge parnassusplein 5 2511 vx den haag     datum 6 november 2017 onderwerp de ervaringen van slachtoffers van kinderpornografie   excellenties,  hierbij wil ik graag het survivors’ survey van het canadian centre for child protection onder uw aandacht brengen. in dit onderzoek is voor het eerst gekeken naar de ervaringen van slachtoffers van kinderpornografie. ik werkte mee aan dit onderzoek, wat ertoe geleid heeft dat bijna de helft van de respondenten nederlands is. alleen al daarom zijn de uitkomsten zeer relevant voor de nederlandse praktijk. ik doe naar aanleiding van dit onderzoek een aantal aanbevelingen, die ik hierna toelicht.  vinden en verwijderen kinderpornografie uit de verhalen van de slachtoffers blijkt dat het misbruik hen blijft achtervolgen omda

...

>209additional tablestable b3.33 nature of the convictions in first instance (20002008)2000 2001 2002 2003 2004 2005 2006 2007 2008 totaln % n % n % n % n % n % n % n % n % n %conviction	for	nonaggravated	human	trafficking   6 9% 15 22% 18 23% 23 20% 30 37% 19 27% 16 22% 27 35% 154 22%conviction	for	aggravated	human	trafficking 61 100% 61 91% 53 78% 61 77% 91 80% 52 63% 51 73% 57 78% 50 65% 537 78%total 61 100% 67 100% 68 100% 79 100% 114 100% 82 100% 70 100% 73 100% 77 100% 691 100%source: pps database (reference date: january 2010)table b3.34 convictions in first instance by sentence imposed (20002008)2000 2001 2002 2003 2004 2005 2006 2007 2008 totaln % n % n % n % n % n % n % n % n % n %nonaggravated	human	traffickingno	principal	punishment     1 7%             1 1%only	a	conditional	sentence     1 7% 1 6%   2 7% 1 5%     5 3%custodial	sentence   6 100% 12 80% 12 67% 20 87% 26 87% 18 95% 15 94% 27 100% 136 88%community	service     1 7% 3 17% 3 13% 2 7%   1 6%   10 6%fine       2 11

>213additional tablestable b3.39  cases dealt with on appeal by most serious registered offence in the indictment (20002004 and 20052008)49most	serious	offence 2000	to	2004 2005	to	2008 totaln % n % n %nonaggravated	human	traffickingconviction 59 95% 57 95% 116 95%acquittal 2 3% 2 3% 4 3%prosecution	declared	inadmissible1 2% 1 2% 2 2%total 62 100% 60 100% 122 100%aggravated	human	traffickingconviction 14 64% 23 82% 37 74%acquittal 6 27% 5 18% 11 22%prosecution	declared	inadmissible2 9%   2 4%total 22 100% 28 100% 50 100%sexual	violenceconviction 14 88% 8 100% 22 92%acquittal 1 6%   1 4%prosecution	declared	inadmissible1 6%   1 4%total 16 100% 8 100% 24 100%other	violenceconviction 2 100% 6 100% 8 100%total 2 100% 6 100% 8 100%other	offencesconviction 14 82% 9 100% 23 88%acquittal 3 18%   3 12%total 17 100% 9 100% 26 100%totalconviction 103 87% 103 93% 206 90%acquittal 12 10% 7 6% 19 8%prosecution declared inadmisisble4 3% 1 1% 5 2%total 119 100% 111 100% 230 100%source: objd (reference

>220trafficking in human beings ten years of independent monitoringtable b3.49 country of birth of suspects (20002008) country	of	birth	of	suspects 2000 2001 2002 2003 2004 2005 2006 2007 2008 totaltop	10n % n % n % n % n % n % n % n % n % n % rafghanistan     1 1%     1 1%       2 0%albania 2 1% 4 3%26(3’)13% 5 3%       5 2% 4 2% 46 3% 10angola   1 1%       1 1% 1 0%     3 0%australia             1 0%   1 0% 2 0%belgium 3 2%   2 1% 1 1% 2 1%   1 0% 1 0% 2 1% 12 1%brazil 1 1% 1 1%   1 1%     1 0% 1 0% 1 0% 6 0%bulgaria  12(3)9%27(2)14%19(3)12%14(4)6% 4 3%13(4’)6%23(3)8%12(3’)6% 124 7% 3cameroon       1 1%       1 0%   2 0%cape	verde   1 1%       2 1%       3 0%china(including	hong	kong)    1 1%     2 1% 2 1% 1 0% 7 3% 13 1% *colombia     1 1%   1 0% 1 1%       3 0%congo	(formerly	zaire)        1 0%     1 0% 1 0% 3 0%(former)	czechoslovakia 3 2% 5 4% 2 1%   2 1% 5 4% 3 1% 1 0% 2 1% 23 1% *dominican	republic               2 1%   2 0%egypt   2 2% 1 1%       1 0% 1 0%   5 0%estonia 2 1% 2 

>225additional tablescountry	of	birth	of	suspects 2000 2001 2002 2003 2004 2005 2006 2007 2008 totaltop	10n % n % n % n % n % n % n % n % n % n % rsaudi	arabia               1 0%   1 0%sierra	leone     1 1% 1 1%     1 0% 5 2% 1 0% 9 1% *somalia     1 1%         1 0% 1 0% 3 0%south	africa               1 0%   1 0%(former)	soviet	union 3 2%6(5)5% 7 4%11(5)7% 8 4% 2 1% 7 3% 1 0% 4 2% 49 3% 9sudan 5 4%   2 1%         1 0%   8 0% *surinam 6 4% 4 3% 5 3% 10 6%9(5’)4%6(4’)4% 11 5% 12 4% 9 4% 72 4% 6syria   1 1%               1 0%thailand     1 1% 2 1%   1 1%   3 1% 1 0% 8 0%tunisia 1 1%               1 0% 2 0%turkey9(4’)6%8(4)6%26(3’)13%16(4)10%24(2)11%15(3)11%18(2)9%28(2)10%10(5)5% 154 9% 2uganda           1 1%       1 0%united	kingdom	         1 0%         1 0%united	states         1 0%         1 0%venezuela             1 0%     1 0%vietnam   1 1%             2 1% 3 0%(former)	yugoslavia9(4’)6%22(2)17% 6 3% 3 2% 3 1% 4 3% 6 3% 1 0% 2 1% 56 3% 8n.	a.	(legal	entity)       1 1% 1 0%   1 0%   2

>a4	 list	of	activities	of	bnrm	in	20091	 interviews,	meetings	and	working	visits	(netherlands)63	641.1	police/royal	netherlands	marechaussee/special	investigative	servicescentre of expertise on human trafficking and migrant smuggling (emm)*national expert group on human trafficking (lem)*operational consultation group on trafficking in human beings (oom)* police academypolice in someren, regional police force for brabant zuidoost1.2	public	prosecution	serviceb9 working group financial, environmental and food safety offences office: chief public prosecutor*financial, environmental and food safety offences office: department of policy and strategynational office: public prosecutor for migrant smuggling and human trafficking *chief public prosecutor, amsterdam*national consultative body of public prosecutors for human trafficking and migrant smuggling *procurator general with the human trafficking portfolio *63 various meetings were held with some of the listed individuals and agencies, 